WGCNA分析
======

基于两个假设：

- 1.相似表达模式的基因可能存在共调控、功能相关或处于同一通路，

- 2.基因网络符合无尺度分布。基于这两点，可以将基因网络根据表达相似性划分为不同的模块进而找出枢纽基因。

1、加载数据
------

In [ ]:
import sys
sys.path.append('C:/ProgramData/Anaconda3/Lib/site-packages')
import rpy2.robjects as robjects

rscript = '''
#加载WGCNA
library(WGCNA)
'''
robjects.r(rscript)

In [ ]:
datapath=""#原始数据矩阵路径
traitpath=""#性状矩阵路径
num_trait='10'#性状个数

In [7]:
#数据展示
import pandas as pd
data=pd.read_csv('')
data.head(10)

,FQ,lab1,lab2,non1,non2
0,F6ZLC6,921.32,0.000000,0.000000,303.367438
1,A0A1L1SQV7,17821.00,6621.523147,4413.130565,5166.555405
2,Q01149,11471.00,46160.629140,5384.415414,11526.570670
3,Q5RIU1,3619.80,7809.348163,1885.105701,2182.818764
4,Q3V0W6,1364.80,0.000000,0.000000,569.733967
5,F6WM66,4368.00,7241.798139,2039.646059,3104.753018
6,Q8CJI4,13524.00,7854.745794,8524.998645,4029.384259
7,Q8K3J1,16105.00,21422.902840,8636.147823,10239.847290
8,Q925N0,4409.30,3973.009462,0.000000,3792.571478
9,Q99LI2,7221.90,10670.513910,4202.559555,4969.762418


In [8]:
#性状数据展示
tr=pd.read_csv('')
tr.head(10)

,type,negstive,target,oppsite,postive,Velocity,latency time,Distance moved Center-point,right sniff Duration,right sniff Frequency,Velocity.1
0,lab1,41.12,118.40,71.12,69.40,10.342773,183,3320.889602,26.04,138,7.253744
1,lab2,20.52,200.88,14.08,64.56,4.129894,60,3014.306856,10.52,50,5.534816
2,non1,3.48,21.72,274.84,0.00,0.301990,300,2365.360851,11.76,43,3.427809
3,non2,132.88,33.80,105.32,28.04,NaN,254,1793.920069,18.16,32,3.942268


In [ ]:
rscript='''
library("openxlsx")
datExpr = read.csv("'''

rs2='''",row.names = 1)
datExpr = t(datExpr)
multiExpr=list(list(data = datExpr))
exprSize = checkSets(multiExpr)
nSets = exprSize$nSets
nGenes = exprSize$nGenes
nSamples = exprSize$nSamples
'''
robjects.r(rscript+datapath+rs2)

In [ ]:
rscript='''
#加载性状
t1=read.csv("'''

rs2='''",row.names = 1)
datTraits=data.frame(t1)
'''
robjects.r(rscript+traitpath+rs2)

2、选择一个β值建立邻接矩阵，使基因分布符合无尺度网络
------

![Fig.1](Fig.1.jpg)

左图：Soft Threshold (power)表示权重，纵坐标表示连接度k与p(k)的相关性。右图：Soft Threshold (power)表示权重，纵坐标表示平均连接度。一般要求k与p(k)的相关性达到0.85时的power作为β值，可以看出本次示例中 β=4

In [ ]:
rs='''
# 设定软阈值范围
powers = c(c(1:10), seq(from = 12, to=20, by=2))
# 获得各个阈值下的 R方 和平均连接度
sft = pickSoftThreshold(multiExpr[[1]]$data, powerVector = powers, verbose = 5)
'''
robjects.r(rs)

In [ ]:
rs='''
# 作图：
sizeGrWindow(9, 5)
par(mfrow = c(1,2));
cex1 = 0.9;
plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
     xlab="Soft Threshold (power)",ylab="Scale Free Topology Model Fit,signed R^2",type="n",
     main = paste("Scale independence"));
text(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
     labels=powers,cex=cex1,col="red");
abline(h=0.90,col="red")
plot(sft$fitIndices[,1], sft$fitIndices[,5],
     xlab="Soft Threshold (power)",ylab="Mean Connectivity", type="n",
     main = paste("Mean connectivity"))
text(sft$fitIndices[,1], sft$fitIndices[,5], labels=powers, cex=cex1,col="red")

softPower <- sft$powerEstimate
adjacency = adjacency(multiExpr[[1]]$data, power = softPower);
TOM = TOMsimilarity(adjacency);
dissTOM = 1-TOM
hierTOM = hclust(as.dist(dissTOM),method="average");

'''
robjects.r(rs)

In [ ]:
robjects.r('dev.off()')

3、检验选定的β下基因网络是否逼近scale free
------

![Fig.1](Fig.2.png)

In [ ]:
rs='''
# ADJ1_cor <- abs(WGCNA::cor( multiExpr[[1]]$data,use = "p" ))^softPower
k <- softConnectivity(datE=multiExpr[[1]]$data,power=softPower) 
sizeGrWindow(10, 5)
par(mfrow=c(1,2))
hist(k)
scaleFreePlot(k,main="Check Scale free topology\n")
'''
robjects.r(rs)

In [ ]:
robjects.r('dev.off()')

4、对刚才得到的拓扑矩阵使用相异度dissimilarity between genes对基因进行聚类，然后使用动态剪切法对树进行剪切成不同的模块（模块最小基因数为30）
------

![Fig.3](Fig.3.png)

In [ ]:
rs='''
#聚类
geneTree = hclust(as.dist(dissTOM), method = "average");
# Plot the resulting clustering tree (dendrogram)
windows()
sizeGrWindow(12,9)
plot(geneTree, xlab="", sub="", main = "Gene clustering on TOM-based dissimilarity",
     labels = FALSE, hang = 0.04);
minModuleSize = 30;
# 动态切割树:
dynamicMods = cutreeDynamic(dendro = geneTree, distM = dissTOM,
                            deepSplit = 2, pamRespectsDendro = FALSE,
                            minClusterSize = minModuleSize);
table(dynamicMods)

'''
robjects.r(rs)

In [ ]:
rs='''

#模块颜色指代
dynamicColors <- labels2colors(dynamicMods)

 
plotDendroAndColors(geneTree, dynamicColors, 'Dynamic Tree Cut',
    dendroLabels = FALSE, addGuide = TRUE, hang = 0.03, guideHang = 0.05,
    main = 'Gene dendrogram and module colors')
table(dynamicColors)
windows()
dev.off()

'''
robjects.r(rs)

In [ ]:
robjects.r('dev.off()')

In [ ]:
denew='''
windows()
dev.off()
'''
robjects.r(denew)

5、计算每个模块的特征向量基因，为某一特定模块第一主成分基因E。代表了该模块内基因表达的整体水平
------

![Fig.4](Fig.6.png)

6、拓扑重叠热图，图中行和列都表示单个基因，深黄色和红色表示高度的拓扑重叠。
------

![Fig.4](Fig.4.png)

热图中，若基因间表达相似度越高，则颜色越深。可以看到，同一模块内的基因具有较高的共表达模式，而不同模块间则相差明显。

In [ ]:
rs='''
windows()
sizeGrWindow(12,9)
#基因表达聚类树和共表达拓扑热图
plot_sim <- -(1-TOM)
plot_sim <- log(TOM)
diag(plot_sim) <- NA
TOMplot(plot_sim, geneTree, dynamicColors,
    main = 'Network heatmap plot, selected genes')
    
#模块特征基因计算
#计算基因表达矩阵中模块的特征基因（第一主成分）
MEList <- moduleEigengenes(multiExpr[[1]]$data, colors = dynamicColors)
MEs <- MEList$eigengenes
head(MEs)[1:6]

#输出模块特征基因矩阵
#write.table(MEs, 'moduleEigengenes.txt', sep = '\t', col.names = NA, quote = FALSE)

plotEigengeneNetworks(MEs, '', cex.lab = 0.8, xLabelsAngle= 90,
    marDendro = c(0, 4, 1, 2),)
windows()
dev.off()
'''
robjects.r(rs)

7、模块与样本性状相关性热图，行表示模块，列表示性状。方块里的值表示相关性和pvalue.
------

![Fig.5](Fig.5.png)

图中，每一行代表不同的基因共表达模块，每一列代表不同的临床表型，数值代表了相关系数，并分别通过红色和蓝色区分正负相关，括号中的值为显著性p值。

由此可帮助评估，哪些模块中的基因可能与特定的生理特征、疾病进展等密切相关。

In [ ]:
rs='''
#热图
moduleTraitCor_noFP <- cor(MEs, datTraits[,1:'''
rs2='''], use = "p");
moduleTraitPvalue_noFP = corPvalueStudent(moduleTraitCor_noFP, nSamples); 
textMatrix_noFP <- paste(signif(moduleTraitCor_noFP, 2), "\n(", signif(moduleTraitPvalue_noFP, 1), ")", sep = ""); 
par(mar = c(10, 8.5, 3, 3)); 
labeledHeatmap(Matrix = moduleTraitCor_noFP, 
               xLabels = names(datTraits[,1:'''
rs3=''']), 
               yLabels = names(MEs), 
               ySymbols = names(MEs), 
               colorLabels = FALSE, 
               colors = blueWhiteRed(50), 
               textMatrix = textMatrix_noFP,
               setStdMargins = FALSE, 
               cex.text = 0.65, 
               zlim = c(-1,1), 
               main = paste("Module-trait relationships")) 
windows()
dev.off()
'''
robjects.r(rs+num_trait+rs2+num_trait+rs3)

In [ ]:
module_name='tan'